In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import vuong_tests2

In [2]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


yn,xn,nobs = gen_data()
print(xn.shape)
print(sm.add_constant(xn).shape)
print(scipy.stats.mode(yn))

(1000,)
(1000, 2)
ModeResult(mode=array([0.]), count=array([87]))


In [3]:
class Tobit(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(Tobit,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        
        beta = params
        
        mu_y = np.matmul(x,beta)
        
        pr_y = stats.norm.logpdf( y, loc = mu_y)
        
       
        #if complete case, assign pr missing to all observations...
        pr_m = stats.norm.logcdf( y, loc = mu_y)
        
        #we're done if ols
        if self.ols:
            return pr_y
        else:
            ll = (1-m)*pr_y + m*pr_m
            return ll
        
    def score(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        m_x = np.repeat(m,x.shape[1]).reshape(x.shape)
        
        if ols: #if OLS use all the data...
            m, m_x = np.ones(y.shape), np.ones(x.shape)
        
        
        b = params

        beta_jac = np.zeros(len(b))
        
        #for censored
        if not cc and not ols: 
            left_stats = (y - np.dot(x, b)) 
            l_pdf = scipy.stats.norm.logpdf(left_stats)
            l_cdf = scipy.stats.norm.logcdf(left_stats)
            left_frac = np.exp(l_pdf - l_cdf)
            beta_left = np.dot(left_frac*m, x*m_x)
            beta_jac -= beta_left
        
        #for non-censored
        mid_stats = (y - np.dot(x, b))
        beta_mid = np.dot(mid_stats*(1-m), x*(1-m_x) )
        beta_jac += beta_mid
        
        # by chain rule, since the expression above is dloglik/dlogsigma
        return beta_jac



model1 =  Tobit(yn,sm.add_constant(xn))
model1_fit = model1.fit(disp=False)
print(model1_fit.summary())

model2 =  Tobit(yn,sm.add_constant(xn),ols=True)
model2_fit = model2.fit(disp=False)
print(model2_fit.summary())

                                Tobit Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1388.2
Model:                          Tobit   AIC:                             2780.
Method:            Maximum Likelihood   BIC:                             2790.
Date:                Mon, 15 Mar 2021                                         
Time:                        22:25:46                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9182      0.048     19.136      0.000       0.824       1.012
x1             0.2600      0.007     38.322      0.0

In [4]:
def setup_shi(yn,xn):
    model1 = Tobit(yn,sm.add_constant(xn))
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)    
    hess1 = model1.hessian(model1_fit.params)
    k1 = len(model1_fit.params)
    
    #fit logistic values
    model2 = Tobit(yn,sm.add_constant(xn),ols=True)
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)    
    hess2 = model2.hessian(model2_fit.params)
    k2 = len(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

# Main examples

In [5]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0.87 0.   0.13], boot1: [0.88 0.   0.12], boot2: [0.88 0.   0.12], llr:-1.752820678100997, std: 3.9640293032879175, omega:0.1766831175688043
[0.98 0.   0.02]


In [6]:
beta0 = 1.
beta1 = .5

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:44.77234801295689, std: 13.170311452765036, omega:0.5177367084661273
[0.11 0.89 0.  ]


In [7]:
beta0 = 1.
beta1 = 1.

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(100,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(100,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:231.05692531384383, std: 48.82757737926294, omega:2.0816340771180886
[0.31 0.69 0.  ]


# Misc examples

In [8]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 1. 0.], boot1: [0. 1. 0.], boot2: [0. 1. 0.], llr:433.93300431796416, std: 65.33835463137629, omega:1.9658972592489856
[0.05 0.95 0.  ]


In [9]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0.1 0.  0.9], boot1: [0.1 0.  0.9], boot2: [0.1 0.  0.9], llr:-154.70540624939417, std: 38.42049071494766, omega:1.184009262693124
[0.15 0.   0.85]


In [10]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

reg: [0. 0. 1.], boot1: [0. 0. 1.], boot2: [0. 0. 1.], llr:-107.23882714589745, std: 9.571812901967146, omega:0.28808331694618433
[0. 0. 1.]


In [11]:
def gen_data(beta0=3.29543322,beta1=0):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
reg,boot1,boot2, llr,std, omega = vuong_tests2.monte_carlo(20,gen_data,setup_shi)
print("reg: %s, boot1: %s, boot2: %s, llr:%s, std: %s, omega:%s"%(reg,boot1,boot2,llr,std, omega))

shi_result = vuong_tests2.monte_carlo_shi(20,setup_shi,gen_data)
print(shi_result)

../vuong_tests2.py:40: RuntimeWarning: invalid value encountered in double_scalars
  test_stat = llr/(omega*np.sqrt(nobs))
../vuong_tests2.py:76: RuntimeWarning: divide by zero encountered in cdouble_scalars
  boot_teststat =  (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
../vuong_tests2.py:77: RuntimeWarning: divide by zero encountered in cdouble_scalars
  boot_teststatc = (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2c*nobs))
../vuong_tests2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  boot_teststat =  (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2*nobs))
../vuong_tests2.py:77: RuntimeWarning: divide by zero encountered in double_scalars
  boot_teststatc = (llr +V.sum()/(2*nobs) )/(np.sqrt(omega2c*nobs))
/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3948: RuntimeWarning: invalid value encountered in multiply
  x1 = take(ap, indices_below, axis=axis) * weights_below
/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/li

reg: [1. 0. 0.], boot1: [1. 0. 0.], boot2: [1. 0. 0.], llr:-0.31546016507033847, std: 0.6780546158134795, omega:0.008774437227382376


../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide


[0.3  0.35 0.35]


../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
../vuong_tests2.py:214: RuntimeWarning: divide by zero encountered in true_divide
  Tnd = (nLR_hat+V.sum()/2)/np.sqrt(n*nomega2_hat + cstar*(V*V).sum())
